In [1]:
import retinanalysis as ra
import retinanalysis.schema as schema
import numpy as np
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt

ra.djconnect()

[2025-07-01 13:02:10,890][INFO]: Connecting root@127.0.0.1:3306
[2025-07-01 13:02:10,932][INFO]: Connected root@127.0.0.1:3306


## Step 1: Query protocol across all experiments

In [2]:
protocols_to_query = ['movingletters', 'presentmatfiles']
experiment_query = ra.get_datasets_from_protocol_names(protocols_to_query)
display(experiment_query)


Found 1 protocols matching "movingletters":
['edu.washington.riekelab.protocols.MovingLetters']

Found 1 protocols matching "presentmatfiles":
['edu.washington.riekelab.protocols.PresentMatFiles']

Found 4 experiments, 6 epoch blocks.



,exp_name,datafile_name,NDF,chunk_name,protocol_name,is_mea,data_dir,group_label,experiment_id,protocol_id,group_id,block_id,chunk_id
0,20250514C,data003,0.5,movies_images,edu.washington.riekelab.protocols.PresentMatFiles,1,20250514C/data003,PresentMatFiles,132,68,2428,3485,563
1,20250514C,data015,0.5,moving_letters,edu.washington.riekelab.protocols.MovingLetters,1,20250514C/data015,DR Moving Letters,132,67,2440,3497,565
2,20250530C,data002,0.5,moving_letters,edu.washington.riekelab.protocols.MovingLetters,1,20250530C/data002,Moving Letters,139,67,2493,3560,595
3,20250429C,data007,0.5,moving_letters,edu.washington.riekelab.protocols.MovingLetters,1,20250429C/data007,DR moving letters,140,67,2501,3572,596
4,20250429C,data009,0.5,moving_letters,edu.washington.riekelab.protocols.MovingLetters,1,20250429C/data009,DR moving letters,140,67,2503,3574,596
5,20250527C,data010,0.5,moving_letters,edu.washington.riekelab.protocols.MovingLetters,1,20250527C/data010,Moving E,147,67,2661,3761,692


## Select experiment and data file of interest

In [ ]:
exp_name = experiment_query.loc[1, 'exp_name']
datafile_name = experiment_query.loc[1,'datafile_name']

# Create response, stim, analysis_chunk and pipeline class
stim = ra.StimBlock(exp_name, datafile_name)
print(stim)
display(stim.df_epochs.loc[0,'epoch_parameters']['NDF'])
display(stim.df_epochs.loc[0,'epoch_parameters']['ndfs'])
display(stim.df_epochs.loc[0,'epoch_parameters'].keys())

StimBlock with properties:
  exp_name: 20250514C
  datafile_name: data015
  chunk_name: moving_letters
  protocol_name: edu.washington.riekelab.protocols.MovingLetters
  noise_protocol_name: manookinlab.protocols.SpatialNoise
  nearest_noise_chunk: chunk3
  parameter_names of length: 57
  df_epochs for 100 epochs



0.5

'["FW10","C1","C2","C5"]'

In [7]:
response = ra.ResponseBlock(exp_name, datafile_name)
print(response)
print(response.df_spike_times)

Loading VCD from /Volumes/data-1/data/sorted/20250514C/data015/kilosort2.5 ...
Fetching EIs...


  0%|          | 0/862 [00:00<?, ?it/s]

ResponseBlock with properties:
  exp_name: 20250514C
  datafile_name: data015
  protocol_name: edu.washington.riekelab.protocols.MovingLetters
  ss_version: kilosort2.5
  n_epochs: 100
  cell_ids of length: 862
  df_spike_times with shape: (862, 1)

                                               spike_times
cell_id                                                   
1        [[61.55, 131.89999999999998, 215.2, 275.05, 32...
3        [[82.19999999999999, 111.94999999999999, 149.7...
5        [[2605.7000000000003, 17261.3], [13740.25, 140...
11       [[], [], [], [], [10228.400000000001], [], [],...
12       [[87.35, 1382.35, 1714.5, 2560.25, 3195.35, 34...
...                                                    ...
1525     [[2090.1, 2988.15, 3460.5, 3886.05, 4342.1, 48...
1526     [[12399.050000000001, 19184.3], [], [751.19999...
1528     [[4490.5, 14833.75, 16454.850000000002, 19574....
1529     [[14919.6, 15882.05, 15971.9, 19002.35, 22312....
1535     [[459.6, 463.34999999999997, 553.

In [9]:
analysis_chunk = ra.AnalysisChunk(exp_name, stim.nearest_noise_chunk)
print(analysis_chunk)
display(analysis_chunk.cell_params_df)

Loading VCD from /Volumes/data-1/analysis/20250514C/chunk3/kilosort2.5 ...
Fetching params...


  0%|          | 0/545 [00:00<?, ?it/s]

Fetching EIs...


  0%|          | 0/545 [00:00<?, ?it/s]

AnalysisChunk with properties:
  exp_name: 20250514C
  chunk_name: chunk3
  ss_version: kilosort2.5
  noise_protocol: manookinlab.protocols.SpatialNoise
  data_files: ['data010']
  typing_files: ['dragos_kilosort2.5.autoClassification.txt']
  numXChecks: 203
  numYChecks: 127
  staXChecks: 142
  staYChecks: 88
  canvas_size: [1824, 1140]
  microns_per_pixel: 3.37
  cell_ids of length: 545
  rf_params with fiels: ['center_x', 'center_y', 'std_x', 'std_y', 'rot']
  cell_params_df of shape: (545, 7)



,cell_id,center_x,center_y,std_x,std_y,rot,typing_file_0
0,1,124.227242,59.448746,3.202960,2.701562,77.642479,OnM
1,2,123.794266,57.000000,2.341889,3.202256,0.000000,OnM
2,4,122.715286,53.013836,4.105193,4.393680,8.706535,OffP
3,5,125.166023,47.067680,4.253757,3.927680,45.932983,OffP
4,6,122.233604,47.615337,2.612151,2.113712,80.780235,OffM
...,...,...,...,...,...,...,...
540,1090,144.025406,67.408958,2.480099,2.481758,50.183739,OffM
541,1091,100.333333,80.666667,5.162364,1.720788,-7.272467,Crap
542,1096,45.684573,20.753444,1.433115,1.000000,-10.206788,Edge
543,1103,31.500000,106.500000,1.000000,1.000000,0.000000,Crap


In [ ]:
pipeline = ra.MEAPipeline(stim, response, analysis_chunk)
print(pipeline)
display(response.df_spike_times)



Ref clusters matched: 349
Ref clusters unmatched: 196
64.04% matched, 35.96% unmatched.
MEAPipeline with properties:
  stim_block and response_block from: MovingLetters
  analysis_chunk: chunk3
  match_dict: with chunk3_id : MovingLetters_id


,spike_times,cell_type
cell_id,,
1,"[[61.55, 131.89999999999998, 215.2, 275.05, 32...",OnM
3,"[[82.19999999999999, 111.94999999999999, 149.7...",OnM
5,"[[2605.7000000000003, 17261.3], [13740.25, 140...",OffM
11,"[[], [], [], [], [10228.400000000001], [], [],...",OffP
12,"[[87.35, 1382.35, 1714.5, 2560.25, 3195.35, 34...",OffP
...,...,...
1525,"[[2090.1, 2988.15, 3460.5, 3886.05, 4342.1, 48...",Unmatched
1526,"[[12399.050000000001, 19184.3], [], [751.19999...",Unmatched
1528,"[[4490.5, 14833.75, 16454.850000000002, 19574....",Unmatched


In [ ]:
display(response.df_spike_times)

In [ ]:
epoch_params = ['imageOrder']
image_order = [lst for lst in stim.df_epochs['imageOrder']]
image_order = np.array(image_order)

# display(stim.df_epochs.columns)

images_per_epoch = stim.df_epochs.loc[0,'epoch_parameters']['imagesPerEpoch']
num_epochs = len(stim.df_epochs)

# epochs_to_see = [1, 2, 5, 245, 6, 232]
# display(stim.df_epochs[['imageOrder', 'matFile']].query('epoch_index == @epochs_to_see'))

cell_of_interest = 6
cell_1_times = response.df_spike_times.query('cell_id == 1')['spike_times'].values

OffP_noise_ids = analysis_chunk.cell_params_df.query('typing_file_0 == "OffP"')['cell_id'].values
OffP_protocol_ids = [pipeline.match_dict[cell] for cell in OffP_noise_ids if cell in pipeline.match_dict.keys()]
OffP_responses = response.df_spike_times.query('cell_id == @OffP_protocol_ids')
OffP_spike_times = OffP_responses['spike_times'].values

test = np.empty((OffP_spike_times.shape[0], OffP_spike_times[0].shape[0]), dtype = object)
for i in range(test.shape[0]):
    test[i,:] = OffP_spike_times[i]

print(test.shape)
fig, ax = plt.subplots()

epochs_to_plot = np.arange(0,9)
ax.eventplot(test[:,0], color = 'k', linewidths = 0.8, linelengths = 0.8)
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Epoch")


In [ ]:
cell_types = ['OnP', 'OffP']
cell_ids = [1, 3, 9, 10, 38, 42, 121,]

cts = list(response.df_spike_times['cell_type'].unique())

display(cts)
test = [key for key, val in pipeline.match_dict.items()]
print(test)


protocol_ids = response.df_spike_times.query('cell_type == @cell_types').index.values
ids_to_plot = [key for key, val in pipeline.match_dict.items() if (val in protocol_ids
                                                           and val in cell_ids)]

corr_ids = [val for key, val in pipeline.match_dict.items() if (val in protocol_ids
                                                        and val in cell_ids)]
print(ids_to_plot)
print(corr_ids)

In [ ]:

def get_ells(analysis_chunk, cell_types, std_scaler = 1.6, units = None):
    
    if units is not None:
        if 'microns' in units.lower():
            scale_factor = analysis_chunk.microns_per_stixel
        elif 'pixels' in units.lower():
            scale_factor = analysis_chunk.pixels_per_stixel
        else:
            scale_factor = 1
    else:
        scale_factor = 1

    ells = []
    for idx, ct in enumerate(cell_types):
        ell_dict = dict()
        cell_df = analysis_chunk.cell_params_df.query('typing_file_0 == @ct')
        rf_params = analysis_chunk.rf_params
        cell_ids = cell_df['cell_id'].values
        for id in cell_ids:
            ell_dict[id] = Ellipse(xy=(rf_params[id]['center_x']*scale_factor,
                                    rf_params[id]['center_y']*scale_factor),
                                    width = rf_params[id]['std_x']*std_scaler*scale_factor,
                                    height = rf_params[id]['std_y']*std_scaler*scale_factor,
                                    angle = rf_params[id]['rot'],
                                    facecolor= f'C{idx}', edgecolor= f'C{idx}',
                                    alpha = 0.7)
        ells.append(ell_dict)
    
    return ells, scale_factor

units = 'pixels'
cell_types = ['OffP', 'OnP', 'OffM', 'OnM', 'SBC']
all_ells, scale_factor = get_ells(analysis_chunk, cell_types, units = units)

fig, ax = plt.subplots(nrows=1, ncols = len(cell_types), figsize = (18, 12/len(cell_types)))

for idx, ct in enumerate(cell_types):
    for id in all_ells[idx]:
        ax[idx].add_patch(all_ells[idx][id])

    ax[idx].set_xlim(0,analysis_chunk.numXChecks * scale_factor)
    ax[idx].set_ylim(0,analysis_chunk.numYChecks * scale_factor)
    if units is not None:
        ax[idx].set_ylabel(units.lower())
        ax[idx].set_xlabel(units.lower())

fig.tight_layout()

